In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.fasttext import FastText

nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

# Unzip and Display Data

In [ ]:
%%capture
!unzip './drive/MyDrive/fake_job_postings.csv' -d 'fake_job_postings'

In [ ]:
df = pd.read_csv("./fake_job_postings/fake_job_postings.csv")
data = df.copy()
data.dropna(inplace=True)
data.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
6,7,Head of Content (m/f),"DE, BE, Berlin",ANDROIDPIT,20000-28000,"Founded in 2009, the Fonpit AG rose with its i...",Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Your Benefits: Being part of a fast-growing co...,0,1,1,Full-time,Mid-Senior level,Master's Degree,Online Media,Management,0
15,16,VP of Sales - Vault Dragon,"SG, 01, Singapore",Sales,120000-150000,Jungle Ventures is the leading Singapore based...,About Vault Dragon Vault Dragon is Dropbox for...,Key Superpowers3-5 years of high-pressure sale...,"Basic: SGD 120,000Equity negotiable for a rock...",0,1,1,Full-time,Executive,Bachelor's Degree,Facilities Services,Sales,0
23,24,"Vice President, Sales and Sponsorship (Busines...","US, CA, Carlsbad",Businessfriend.com,100000-120000,"WDM Group is an innovative, forward thinking d...",#URL_eda2500ddcedb60957fcd7f5b164e092966f8c4e8...,"Job Requirements:A reputation as a ""go-getter""...",Businessfriend will offer a competitive six fi...,0,1,0,Full-time,Executive,Unspecified,Internet,Sales,0
98,99,IC&E Technician,"US, , Stocton, CA",Oil & Energy,95000-115000,...,"IC&amp;E Technician | Bakersfield, CA Mt. Poso...","QualificationsKnowledge, Skills &amp; Abilitie...",BENEFITSWhat is offered:Competitive compensati...,0,1,1,Full-time,Mid-Senior level,High School or equivalent,Oil & Energy,Other,1
102,103,Marketing Administrator,"GB, WAR, Coventry",Marketplace,15000-18000,Renewable Energy and Environmental Protection ...,The job is to support the growth of the #URL_9...,"Computer literateAble to work with HTML, altho...",For a suitably motivated and success orientate...,1,1,0,Full-time,Entry level,Bachelor's Degree,Internet,Marketing,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15185 non-null  object
 8   benefits             10670 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

# N-grams


In [ ]:
def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', str(text))
    return cleaned_text

def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    # Join tokens back into a single string
    preprocessed_text = ' '.join(lemmatized_tokens)

    return preprocessed_text

text_columns = ['title', 'location', 'department', 'salary_range', 'company_profile', 'description', 'requirements', 'benefits', 'employment_type', 'required_experience', 'required_education', 'industry', 'function']

X = 'Title: ' + data['title'] + ' Description: ' + data['description'] + ' Location: ' + data['location'] + \
' Department: ' + data['department'] + ' Company Profile: ' + data['company_profile'] \
+ ' Requirements: ' + data['requirements'] + ' Employment Type: ' + data['employment_type'] \
+ ' Benefits: ' + data['benefits'] + ' Required Experience: ' + data['required_experience'] \
+ ' Industry: ' + data['industry'] + ' Required Education: ' + data['required_education'] \
+ ' Function: ' + data['function'] + ' Salaary Range: ' + data['salary_range']

y = data['fraudulent']

X_train, X_test, y_train, y_test = train_test_split(X.apply(clean_text).apply(preprocess_text), y, test_size=0.25, random_state=11)

# Define a CountVectorizer to extract n-grams
vectorizer = CountVectorizer(ngram_range=(1, 2))  # Use unigrams and bigrams

# Fit the vectorizer on the training data and transform the training data into n-gram features
X_train_ngrams = vectorizer.fit_transform(X_train)

# Transform the test data into n-gram features (using the same vocabulary as the training data)
X_test_ngrams = vectorizer.transform(X_test)

# Train a logistic regression model
clf = LogisticRegression()
clf.fit(X_train_ngrams, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test_ngrams)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

(774,) (774,)
Accuracy: 0.9948453608247423
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       168
           1       1.00      0.96      0.98        26

    accuracy                           0.99       194
   macro avg       1.00      0.98      0.99       194
weighted avg       0.99      0.99      0.99       194



# Naive Bayes

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np

# Tokenize text data
X = [''.join(sent) for sent in X]

# Initialize a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the TF-IDF vectorizer on the tokenized text data
X_tfidf = vectorizer.fit_transform(X)

# Convert labels to a NumPy array
y = np.array(y)

# Split the data into training and testing sets
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.25, random_state=11)

# Train a Multinomial Naive Bayes model
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.865979381443299
Classification Report:
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       168
           1       0.00      0.00      0.00        26

    accuracy                           0.87       194
   macro avg       0.43      0.50      0.46       194
weighted avg       0.75      0.87      0.80       194



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Define a CountVectorizer to create a binary representation of the text data (presence or absence of each word)
vectorizer = CountVectorizer(binary=True)  # Use binary representation

# Fit the vectorizer on the training data and transform the training data into binary features
X_train_binary = vectorizer.fit_transform(X_train)

# Transform the test data into binary features (using the same vocabulary as the training data)
X_test_binary = vectorizer.transform(X_test)

# Train a Multinomial Naive Bayes model
clf = MultinomialNB()
clf.fit(X_train_binary, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test_binary)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9845360824742269
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       168
           1       1.00      0.88      0.94        26

    accuracy                           0.98       194
   macro avg       0.99      0.94      0.96       194
weighted avg       0.98      0.98      0.98       194



### CountVectorizer and TfidfVectorizer represent text data differently:

CountVectorizer: It represents each document as a bag of words, where each word's frequency is counted. The resulting vector has the length of the vocabulary, and each element corresponds to the frequency of a specific word in the document.
TfidfVectorizer: It also represents each document as a bag of words. However, instead of just counting the frequency of each word, it calculates the Term Frequency-Inverse Document Frequency, which gives more weight to words that are important in a particular document but less common in the entire corpus.
In some cases, CountVectorizer might capture more relevant information for the specific problem, leading to better performance. For example, if the dataset contains text where the raw frequency of words is more important than their relative importance within the document or across the corpus, CountVectorizer might outperform TfidfVectorizer.

However, in many cases, TfidfVectorizer tends to perform better because it reduces the impact of common words and highlights the importance of rare but meaningful words. It is always a good idea to experiment with both methods and compare their performance on your specific dataset.

Word2Vec, BERT, TF-IDF Vectorizer, and CountVectorizer all have the same goal of transforming text data into numerical vectors. The main difference between these methods lies in the way they represent the text data and the information they capture.

CountVectorizer: This method represents a document as a bag of words, where each word's frequency is counted. The resulting vector has the length of the vocabulary, and each element corresponds to the frequency of a specific word in the document.
TF-IDF Vectorizer: Similar to CountVectorizer, TF-IDF Vectorizer also represents a document as a bag of words. However, instead of just counting the frequency of each word, it calculates the Term Frequency-Inverse Document Frequency, which gives more weight to words that are important in a particular document but less common in the entire corpus.
Word2Vec: Word2Vec generates dense, low-dimensional vectors for each word in the vocabulary. These vectors capture semantic relationships between words, such as similarity, analogy, and context. Word2Vec representations can be averaged or combined using other methods to represent an entire document.
BERT: BERT (Bidirectional Encoder Representations from Transformers) is a more recent and advanced technique for generating contextualized word embeddings. Unlike Word2Vec, BERT captures the meaning of a word based on its surrounding context, producing more nuanced representations. BERT embeddings can also be averaged or combined to represent an entire document.
In summary, all these methods aim to transform text data into numerical vectors, but they differ in the way they represent words and documents and the information they capture.

## Perplexity

In [ ]:
# Get the predicted probabilities for each class
y_prob = clf.predict_proba(X_test_binary)

# Calculate cross-entropy loss (negative log-likelihood)
cross_entropy = -np.log(y_prob[np.arange(len(y_test)), y_test])

# Calculate perplexity
perplexity = np.exp(np.mean(cross_entropy))
print("Perplexity:", perplexity)

In [ ]:
real_job_post = "Engineering Manager, Data Privacy - Facebook US, CA, Menlo Park Engineering At Facebook, we're bringing the world closer together. Through our family of apps and services, we're building a different kind of company that connects billions of people around the world, gives them ways to share what matters most to them, and helps bring people closer together. Whether we're creating new products or helping a small business expand its reach, people at Facebook are builders at heart. Our global teams are constantly iterating, solving problems, and working together to empower people around the world to build community and connect in meaningful ways. Together, we can help people build stronger communities — we're just getting started. Facebook is seeking an experienced Engineering Manager to join our Data Privacy team. The Data Privacy team is responsible for building systems and infrastructure to protect user data and ensure compliance with privacy regulations. In this role, you will lead a team of talented engineers, collaborate with cross-functional partners, and drive initiatives to enhance data privacy across Facebook's products and services. Responsibilities: - Lead and mentor a team of engineers, providing technical guidance, support, and feedback - Collaborate with product managers, designers, and other stakeholders to define and prioritize features related to data privacy - Design, implement, and maintain scalable and reliable systems and infrastructure for data privacy - Drive initiatives to improve data protection, privacy compliance, and user trust - Stay informed about privacy regulations and industry best practices, and incorporate them into Facebook's data privacy strategy Requirements: - Bachelor's degree in Computer Science, Engineering, or a related field - 5+ years of experience in software engineering, with at least 2 years in a leadership role - Strong technical background in building and maintaining large-scale distributed systems - Experience with data privacy, security, or compliance is a plus - Excellent communication and interpersonal skills, with the ability to collaborate effectively with cross-functional teams - Passion for privacy and user trust, with a commitment to protecting user data and upholding high ethical standards at Facebook If you're passionate about data privacy and want to work on cutting-edge technology with a global impact, we'd love to hear from you. Apply now and join us in our mission to bring the world closer together."

# Preprocess the sample job post text
processed_sample_post = preprocess_text(real_job_post)

# Transform the preprocessed text using the TF-IDF vectorizer
X_sample = tfidf_vectorizer.transform([processed_sample_post])

# Predict whether the sample job post is fraudulent or not
fraudulent_prediction = nb_classifier.predict(X_sample)

# Print the prediction
print("Predicted label for the sample job post:", "Fraudulent" if fraudulent_prediction[0] == 1 else "Not Fraudulent")


Predicted label for the sample job post: Not Fraudulent


In [ ]:
import gensim.downloader as api
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load pre-trained Word2Vec embeddings
word2vec_model = api.load('word2vec-google-news-300')

# Function to convert text to a fixed-length vector representation
def text_to_vector(text):
    words = text.split()
    vectors = [word2vec_model[word] for word in words if word in word2vec_model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Convert text data to vector representations
X = np.vstack([text_to_vector(text) for text in data['combined_text']])
y = data['fraudulent']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the logistic regression classifier
logreg_classifier = LogisticRegression(max_iter=1000)

# Train the classifier
logreg_classifier.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = logreg_classifier.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))


[==================================================] 100.0% 1662.8/1662.8MB downloaded
Accuracy: 0.9161290322580645
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       142
           1       0.00      0.00      0.00        13

    accuracy                           0.92       155
   macro avg       0.46      0.50      0.48       155
weighted avg       0.84      0.92      0.88       155



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn.decomposition import PCA

# Assuming 'text_columns' and 'data' are defined earlier in your code
text_columns = ['title', 'location', 'department', 'salary_range', 'company_profile', 'description', 'requirements', 'benefits', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education', 'industry', 'function']
X = data[text_columns] # Features
y = data.fraudulent # Target variable

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)


# Assuming 'description' is one of your text columns
sentences = [sentence.split() for sentence in X_train['description']]  # Split each description into words
for sentence in X_train['description']:
  break
# Train Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Function to transform each sentence into an average vector of its word embeddings
def word_averaging(model, sentence):
    words = sentence.split()
    return np.mean([model.wv[word] for word in words if word in model.wv] or [np.zeros(model.vector_size)], axis=0)

# Transforming each description into an average word embedding vector
X_train_word2vec = np.array([word_averaging(word2vec_model, sentence) for sentence in X_train['description']])
X_test_word2vec = np.array([word_averaging(word2vec_model, sentence) for sentence in X_test['description']])

# Now X_train_word2vec and X_test_word2vec contain the word embedding representations of your text data

# Optionally, you can reduce dimensionality using PCA
pca = PCA(n_components=50)
X_train_word2vec_pca = pca.fit_transform(X_train_word2vec)
X_test_word2vec_pca = pca.transform(X_test_word2vec)


# Now X_train_word2vec_pca and X_test_word2vec_pca contain the reduced dimensional representations

# Instantiate the logistic regression model
logreg = LogisticRegression(random_state=16)

# Fit the model with data
logreg.fit(X_train_word2vec_pca, y_train)

# Predict on the test set
y_pred = logreg.predict(X_test_word2vec_pca)

# Compute confusion matrix
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

# Now you can use cnf_matrix or other metrics to evaluate the performance of your model
cnf_matrix

Streaming output truncated to the last 5000 lines.
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str

AttributeError: 'float' object has no attribute 'split'

In [ ]:
data.dropna(inplace=True)
# Assuming 'text_columns' and 'data' are defined earlier in your code
text_columns = ['title', 'location', 'department', 'salary_range', 'company_profile', 'description', 'requirements', 'benefits', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education', 'industry', 'function']
X = data[text_columns] # Features
y = data.fraudulent # Target variable

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the text columns in your training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['description'])

# Transform the text columns in your test data
X_test_tfidf = tfidf_vectorizer.transform(X_test['description'])

# Concatenate the TF-IDF transformed text columns with other numerical columns, if any
# You may need to handle other non-textual columns differently (e.g., one-hot encoding for categorical variables)

# Now, X_train_tfidf and X_test_tfidf contain the TF-IDF transformed features
# Instantiate the logistic regression model
logreg = LogisticRegression(random_state=16)

# Fit the model with data
logreg.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = logreg.predict(X_test_tfidf)

# Compute confusion matrix
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

# Now you can use cnf_matrix or other metrics to evaluate the performance of your model
cnf_matrix

from sklearn.metrics import classification_report
target_names = ['not fraudulent', 'fraudulent']
print(classification_report(y_test, y_pred, target_names=target_names))


                precision    recall  f1-score   support

not fraudulent       0.93      1.00      0.96       178
    fraudulent       1.00      0.19      0.32        16

      accuracy                           0.93       194
     macro avg       0.97      0.59      0.64       194
  weighted avg       0.94      0.93      0.91       194



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Assuming 'df' is your DataFrame with columns representing title, description, department, benefits, etc.
# Replace 'text_columns' with the actual names of your text columns
text_columns = ['title', 'location', 'department', 'salary_range', 'company_profile', 'description', 'requirements', 'benefits', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education', 'industry', 'function']
X_text = data[text_columns].apply(lambda x: ' '.join(map(str, x.dropna())), axis=1)
y = data['fraudulent']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_text, y, test_size=0.2, random_state=42)

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

# Fit and transform the text data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize the Naïve Bayes classifier
naive_bayes_classifier = MultinomialNB()

# Train the classifier
naive_bayes_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the testing set
y_pred = naive_bayes_classifier.predict(X_test_tfidf)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))


KeyboardInterrupt: 